In [1]:
import time
import itertools
from concurrent.futures import ThreadPoolExecutor
import speaker_buddy as buddy

def split_list(a_list, chunks):
    # return (itertools.islice(a_list, i,len(a_list), chunks) for i in range(chunks))
    return (a_list[i::chunks] for i in range(chunks))


def is_prime(n):
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for divisor in range(3, n, 2):
        if n % divisor == 0:
            return False
    return True


def find_next_prime(n):
    for val in itertools.count(n + 1):
        if is_prime(val):
            return val

In [2]:
is_prime(19)

True

In [3]:
find_next_prime(19)

23

In [4]:
with open('../data/prime_mixture.txt') as fp:
    numbers = [int(n.strip()) for n in fp.read().split() if n]

In [5]:
numbers

[15492781, 15492787, 15492803, 15492811, 15492810, 15492833, 15492859, 1549289]

## `multiprocessing`

In [6]:
from multiprocessing import Pool

In [7]:
with buddy.measure_time() as tm:
    results = search_primes_single_thread(numbers)

In [7]:
with Pool(5) as p:
    result = p.map(is_prime, numbers)

In [8]:
result

[True, True, True, True, False, True, True, False]

Measuring time for subprocess:

In [9]:
with buddy.measure_time() as tm:
    with Pool(5) as p:
        result = p.map(is_prime, numbers)

In [10]:
tm.print()

Total duration: 3.65 seconds


There's a slight improvement.

## `concurrent.futures`

In [11]:
from concurrent.futures import ProcessPoolExecutor

In [12]:
with ProcessPoolExecutor() as executor:
    result = executor.map(is_prime, numbers)

In [14]:
list(result)

[True, True, True, True, False, True, True, False]

Comparing time:

In [15]:
with buddy.measure_time() as tm:
    with ProcessPoolExecutor() as executor:
        result = executor.map(is_prime, numbers)

In [16]:
tm.print()

Total duration: 2.42 seconds
